In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,BatchNormalization,Bidirectional,LSTM,Dropout,Conv2D
from tensorflow.keras.models import Model,load_model,save_model,Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.applications.resnet import ResNet152
from PIL import Image
import glob
import tqdm
import pickle

In [4]:
class Dataloader:
    def __init__(self, datapath, classes, max_frames, img_shape, channels, saving_dir):
        self.datapath = datapath
        self.classes = classes
        self.seq_length = max_frames
        self.height = img_shape[0]
        self.width = img_shape[1]
        self.channels = channels
        self.model = ResNet152(include_top = False,input_shape=(224,224,3),weights = 'imagenet')
        
        for layer in self.model.layers:
            layer.trainable = False
            
    def get_frame_sequence(self,path):
        total_frames = os.listdir(path)
        arr = np.zeros((224,224,3,40))
        if len(total_frames) >= 160:
            counter = 0
            for i in range(1,160,4):
                x = Image.open(os.path.join(path,str(i) + '.JPG'))
                x = np.array(x)
                arr[:,:,:,counter] = x
                counter+=1
                if(counter >= self.seq_length):
                    break
                    
        elif((len(total_frames) >= 120) and (len(total_frames) <160)):
            counter = 0
            for i in range(1,120,3):
                x = Image.open(os.path.join(path,str(i) + '.JPG'))
                x = np.array(x)
                arr[:,:,:,counter] = x
                counter+=1
                if(counter >= self.seq_length):
                    break
        
        elif((len(total_frames) >= 99) and (len(total_frames) < 120)):
            counter = 0
            for i in range(0,40,2):
                x = Image.open(os.path.join(path,str(i) + '.JPG'))
                x = np.array(x)
                arr[:,:,:,counter] = x
                counter+=1
            for i in range(41,99,3):
                x = Image.open(os.path.join(path,str(i) + '.JPG'))
                x = np.array(x)
                arr[:,:,:,counter] = x
                counter += 1
                if(counter >= self.seq_length):
                    break
                
        elif((len(total_frames) >= 80) and (len(total_frames) < 98)):
            counter = 0
            for i in range(0,80,2):
                x = Image.open(os.path.join(path,str(i) + '.JPG'))
                x = np.array(x)
                arr[:,:,:,counter] = x
                counter += 1
                if(counter == self.seq_length):
                    break
        else:
            counter = 0
            for i in range(40):
                x = Image.open(os.path.join(path,str(i) + '.JPG'))
                x = np.array(x)
                arr[:,:,:,counter] = x
                counter += 1
                if(counter >= self.seq_length):
                    break
                    
        #print(arr.shape)
        return arr
            
    def extract_feature(self,x_train):
        x_op = np.zeros((2048,40))
        for i in range(x_train.shape[3]):
            x = self.model.predict(x_train[:,:,:,i])
            x_op[:,i] = x
        
        return x_op
    
    def get_all_sequence_in_memory(self):
        counter = 0
        y_train = []
        for i in self.classes:
            directory_path = os.path.join(self.datapath,i)
            if i == 'violence':
                y = 1
            else:
                y = 0
                
            list_dir = os.listdir(directory_path)
            x_train = []
            for folder in tqdm.tqdm(list_dir):
                path = os.path.join(directory_path,folder)
                arr = self.get_frame_sequence(path)
                x_ext = self.extract_features(arr)
                x_train.append(x_ext)
                counter+=1
                y_train.append(y)
        save_file_x = os.path.join(saving_dir,'data_x_ext.npy')
        save_file_y = os.path.join(saving_dir,'data_y,npy')
        np.save(save_file_x,np.array(x_train))
        np.save(save_file_y,np.array(y_train))
        return x_train,y_train
    
    def load_npy_file(self):
        x_train = np.load(self.datapath + 'data_x.npy')
        y_train = np.load(self.datapath + 'data_y.npy')
        
        return x_train,y_train
    
    def extract_features(self,x_train):     #x_train must be numpy array
        model = ResNet152(include_top = False,image_shape=(224,224,3),weights = 'imagenet')
        features = []
        for layer in model.layers:
            layer.trainable = False
        total_videos = 2000
        for i in range(2000):
            temp = np.zeros((224,224,3,40))
            for j in range(40):
                fv = model.predict(x_train[:,:,:,j,i])
                temp[:,:,:,j] = fv
            features.append(temp)
        np.save('',np.array(features))

In [11]:
def load_data():
    data_loader = Dataloader(r'images',['violence','non_violence'],40,(224,224),3,r'images\saving_things')
    x_val,y_val = data_loader.get_all_sequence_in_memory()

In [13]:
if __name__ == '__main__':
    load_data()

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]


ValueError: could not broadcast input array from shape (126,224,3) into shape (224,224,3)

In [9]:
import sys
import os
import cv2
import imutils
from imutils.video import FPS
import numpy as np
from threading import Thread
import time
from tqdm import tqdm

if sys.version_info >= (3, 0):
    from queue import Queue

else:
    from Queue import Queue
    
class FileVideoStream:
    def __init__(self,path,queueSize=128):
        self.stream = cv2.VideoCapture(path)
        self.stopped = False
        self.Q = Queue(maxsize = queueSize)
        
    def start(self):
        t = Thread(target=self.update,args=())
        t.daemon = True
        t.start()
        return self

    def update(self):
        while True:
            if self.stopped:
                return
            if not self.Q.full():
                (grabbed, frame) = self.stream.read()
            
                if not grabbed:
                    self.stop()
                    return
            
                self.Q.put(frame)

    def read(self):
        return self.Q.get()

    def more(self):
        return self.Q.qsize() > 0

    def stop(self):
        self.stopped = True


In [12]:
data_path = r"Real Life Violence Dataset/NonViolence"
save_path = "images/non_violence"

In [13]:
video_list = os.listdir(data_path)
print(video_list)

for video in tqdm(video_list):
    video_path = os.path.join(data_path,video)
    name = video.split('.')[0]
    saving_at = os.path.join(save_path,name)
    if not os.path.exists(saving_at):
        os.makedirs(saving_at)
    
    fvs = FileVideoStream(video_path).start()
    time.sleep(1)  #allowing buffer to fill
    
    counter = 0
    while fvs.more():
        frame = fvs.read()
        frame = cv2.resize(frame,(224,224),interpolation = cv2.INTER_NEAREST)
        frame_name = os.path.join(saving_at,str(counter)+'.JPG')
        cv2.imwrite(frame_name,frame)
        counter+=1

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

['NV_1.mp4', 'NV_10.mp4', 'NV_100.mp4', 'NV_1000.mp4', 'NV_101.mp4', 'NV_102.mp4', 'NV_103.mp4', 'NV_104.mp4', 'NV_105.mp4', 'NV_106.mp4', 'NV_107.mp4', 'NV_108.mp4', 'NV_109.mp4', 'NV_11.mp4', 'NV_110.mp4', 'NV_111.mp4', 'NV_112.mp4', 'NV_113.mp4', 'NV_114.mp4', 'NV_115.mp4', 'NV_116.mp4', 'NV_117.mp4', 'NV_118.mp4', 'NV_119.mp4', 'NV_12.mp4', 'NV_120.mp4', 'NV_121.mp4', 'NV_122.mp4', 'NV_123.mp4', 'NV_124.mp4', 'NV_125.mp4', 'NV_126.mp4', 'NV_127.mp4', 'NV_128.mp4', 'NV_129.mp4', 'NV_13.mp4', 'NV_130.mp4', 'NV_131.mp4', 'NV_132.mp4', 'NV_133.mp4', 'NV_134.mp4', 'NV_135.mp4', 'NV_136.mp4', 'NV_137.mp4', 'NV_138.mp4', 'NV_139.mp4', 'NV_14.mp4', 'NV_140.mp4', 'NV_141.mp4', 'NV_142.mp4', 'NV_143.mp4', 'NV_144.mp4', 'NV_145.mp4', 'NV_146.mp4', 'NV_147.mp4', 'NV_148.mp4', 'NV_149.mp4', 'NV_15.mp4', 'NV_150.mp4', 'NV_151.mp4', 'NV_152.mp4', 'NV_153.mp4', 'NV_154.mp4', 'NV_155.mp4', 'NV_156.mp4', 'NV_157.mp4', 'NV_158.mp4', 'NV_159.mp4', 'NV_16.mp4', 'NV_160.mp4', 'NV_161.mp4', 'NV_162.mp4',

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [40:18<00:00,  2.42s/it]


In [14]:
data_path = r"Real Life Violence Dataset/Violence"
save_path = "images/violence"

In [ ]:
video_list = os.listdir(data_path)
print(video_list)

for video in tqdm(video_list):
    video_path = os.path.join(data_path,video)
    name = video.split('.')[0]
    saving_at = os.path.join(save_path,name)
    if not os.path.exists(saving_at):
        os.makedirs(saving_at)
    
    fvs = FileVideoStream(video_path).start()
    time.sleep(1)  #allowing buffer to fill
    
    counter = 0
    while fvs.more():
        frame = fvs.read()
        frame = cv2.resize(frame,(224,224),interpolation = cv2.INTER_NEAREST)
        frame_name = os.path.join(saving_at,str(counter)+'.JPG')
        cv2.imwrite(frame_name,frame)
        counter+=1

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

['V_1.mp4', 'V_10.mp4', 'V_100.mp4', 'V_1000.mp4', 'V_101.mp4', 'V_102.mp4', 'V_103.mp4', 'V_104.mp4', 'V_105.mp4', 'V_106.mp4', 'V_107.mp4', 'V_108.mp4', 'V_109.mp4', 'V_11.mp4', 'V_110.mp4', 'V_111.mp4', 'V_112.mp4', 'V_113.mp4', 'V_114.mp4', 'V_115.mp4', 'V_116.mp4', 'V_117.mp4', 'V_118.mp4', 'V_119.mp4', 'V_12.mp4', 'V_120.mp4', 'V_121.mp4', 'V_122.mp4', 'V_123.mp4', 'V_124.mp4', 'V_125.mp4', 'V_126.mp4', 'V_127.mp4', 'V_128.mp4', 'V_129.mp4', 'V_13.mp4', 'V_130.mp4', 'V_131.mp4', 'V_132.mp4', 'V_133.mp4', 'V_134.mp4', 'V_135.mp4', 'V_136.mp4', 'V_137.mp4', 'V_138.mp4', 'V_139.mp4', 'V_14.mp4', 'V_140.mp4', 'V_141.mp4', 'V_142.mp4', 'V_143.mp4', 'V_144.mp4', 'V_145.mp4', 'V_146.mp4', 'V_147.mp4', 'V_148.mp4', 'V_149.mp4', 'V_15.mp4', 'V_150.mp4', 'V_151.mp4', 'V_152.mp4', 'V_153.mp4', 'V_154.mp4', 'V_155.mp4', 'V_156.mp4', 'V_157.mp4', 'V_158.mp4', 'V_159.mp4', 'V_16.mp4', 'V_160.mp4', 'V_161.mp4', 'V_162.mp4', 'V_163.mp4', 'V_164.mp4', 'V_165.mp4', 'V_166.mp4', 'V_167.mp4', 'V_168

 61%|█████████████████████████████████████████████▌                             | 607/1000 [1:30:30<9:57:33, 91.23s/it]